# Collision Avoidance - AlexNetでの自動走行

このnotebookでは、JetBotが「free（直進する）」か「blocked（旋回する）」かを検出する学習済みモデルを使用して、JetBotの衝突回避動作を確認できます。

## 学習済みモデルの読み込み

``train_model_JP.ipynb``ノートブックの指示に従って、すでに``best_model.pth``が存在している事を前提とします。

Pytorch modelの初期化を下記コードでおこないます。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch
import torchvision

########################################
# PyTorchで提供されているAlexNetモデルを読込みます。
# ここでは未学習のモデル構造だけを読み込みます。
########################################
model = torchvision.models.alexnet(pretrained=False)

########################################
# モデルの出力層をJetBotの衝突回避モデル用に置き換えます。
########################################
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 2)

``train_model_JP.ipynb``で学習した``best_model.pth``から学習の重みづけをロードします。

In [ ]:
########################################
# 未学習のモデルに学習結果の重みづけを読込みます。
########################################
model.load_state_dict(torch.load('best_model.pth'))

デフォルトではモデルのweightの計算はCPUで処理されるため、GPUを利用するようにモデルを設定します。

In [ ]:
########################################
# GPU処理が可能な部分をGPUで処理するように設定します。
########################################
device = torch.device('cuda')
model = model.to(device)

## カメラ画像の前処理作成
モデルを読み込みましたが、まだ少し問題があります。  
学習時の入力画像フォーマットと、OpenCVのカメラ画像フォーマットは一致しません。  
これを解消するために、 いくつかの前処理を行う必要があります。これらは、下記の手順になります。

1. カメラ画像をBGRフォーマットからRGBフォーマットに変換します。
> 学習時のjpeg画像はtorchvision.datasets.ImageFolderによって読み込まれます。ImageFolderはPILライブラリを利用して画像ファイルを読み込んだ後、RGBフォーマットに変換しています。このため、学習時の入力画像データはRGBフォーマットになっています。しかしカメラ画像を取得するために使っているOpenCVはデフォルトでBGRフォーマットとなるため、このまま予測すると画像の赤色と青色が入れ替わっているために精度が悪くなります。そこで、カメラ画像を学習時のフォーマットに合わせるためにRGBフォーマットに変換します。
2. HWCをCHWに変換します。
> cudnnはHWC(Height x Width x Channel)をサポートしません。そのため画像情報の並び順をHWCからCHW(Channel x Height x Width)に変換します。
3. カメラ画像を正規化します。
> トレーニング中に使ったのと同じパラメータ（平均と標準偏差）を利用してカメラ画像の各チャンネル(RGB)を正規化します。  
> OpenCVで取得したカメラ画像の1ピクセルはRGBをそれぞれint型で[0, 255]の範囲で表したものになります。  
> しかし、学習時はImageFolderを使ってjpeg画像を読込み、それをtransforms.ToTensor()を使ってTensor型に変換しています。この時、CHWへの変換と計算グラフレイヤの追加の他に、RGB値がint型の[0, 255]からfloat型の[0.0, 1.0]にスケーリングされています。学習時はこの[0.0, 1.0]の値に対してtransforms.Normalize()を行うことでRGB各値を正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリング）しています。 
> ここではToTensor()を使わずにCHWへの変換をおこなっています。そしてImageNetと同じ範囲に各チャンネルをスケーリングするために、Normalize()に渡すパラメータに255.0を掛けています。
4. カメラ画像をGPUメモリに転送します。
> 入力データはモデルと同じデバイスに存在する必要があります。
5. 入力画像データを配列に変更します。
> 学習時はバッチサイズ分の画像を配列にして入力層に与えています。モデルの入力層は学習ために入力データを可変長の配列で受け取る構造になっています。そのため予測時は1枚の画像であっても入力画像データを配列にする必要があります。

* ImageFolderリファレンス：
  * https://pytorch.org/docs/stable/torchvision/datasets.html#imagefolder
* ImageFolder実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/datasets/folder.py
* Normalizeリファレンス：
  * https://pytorch.org/docs/stable/torchvision/transforms.html
* Normalize実装コード：
  * https://github.com/pytorch/vision/blob/master/torchvision/transforms/transforms.py
* 正規化パラメータの値の理由：
  * https://stackoverflow.com/questions/58151507/why-pytorch-officially-use-mean-0-485-0-456-0-406-and-std-0-229-0-224-0-2
* 正規化に意味があるのかどうか：
  * https://teratail.com/questions/234027

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import cv2  # OpenCVライブラリを利用します。
import numpy as np  # 数値計算ライブラリのnumpyを利用します。

########################################
# この値はpytorch ImageNetの学習に使われた正規化（ImageNetデータセットのRGB毎に平均を0、標準偏差が1になるようにスケーリングすること）のパラメータです。
# カメラ画像はこの値でRGBを正規化することが望ましいでしょう。
# ここではtransforms.ToTensor()を使っていないため、正規化前のRGB値の範囲は[0, 255]です。
# そこで、学習時のRGB各範囲と同じ範囲にスケーリングするように正規化パラメータに255.0を掛けて設定します。
########################################
mean = 255.0 * np.array([0.485, 0.456, 0.406])  # 平均のパラメータを定義します。
stdev = 255.0 * np.array([0.229, 0.224, 0.225])  # 標準偏差のパラメータを定義します。

normalize = torchvision.transforms.Normalize(mean, stdev)  # 正規化を定義します。

########################################
# カメラ画像をモデル入力用データに変換します。
########################################
def preprocess(camera_value):
    # preprocess()関数はメインスレッドで動作します。このglobal宣言された変数は同じメインスレッドで定義されているため、省略可能です。
    # preprocess()関数をサブスレッドから呼び出す場合はglobal宣言が必要になります。
    global device, normalize
    # OpenCVで取得したカメラ画像を変数xにコピーします。
    x = camera_value
    # 学習時の画像データはtorchvision.datasets.ImageFolderを使って読み込んでいるため、モデルはRGBフォーマットの画像で学習しています。
    # カメラ映像はOpenCVで読み込んでいるため画像はBGRフォーマットになっています。これをRGBフォーマットに変換します。
    x = cv2.cvtColor(x, cv2.COLOR_BGR2RGB)
    # 画像フォーマットをHWCからCHWに変換します。
    x = x.transpose((2, 0, 1))
    # float32に変換します。
    x = torch.from_numpy(x).float()
    # 正規化します。
    x = normalize(x)
    # GPUデバイスを利用します。
    x = x.to(device)
    # バッチ配列に変換します。
    x = x[None, ...]
    # 入力用データxを返します。
    return x

> すばらしい、これでカメラ画像をニューラルネットワークの入力フォーマットに変換するための、pre-processing関数を定義できました。　

## カメラの動作確認
それでは、カメラを起動して表示しましょう。  
JetBotが「blocked」（旋回する）と判断している確率を表示するためのスライダーも用意します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import traitlets  # カメラ画像などのデータが更新されたときに、連動して処理を実行させるためにtraitletsライブラリを利用します。
from IPython.display import display  # ウィジェットを表示するためのdisplayライブラリを利用します。
import ipywidgets.widgets as widgets  # Jupyter標準のウィジェットを利用します。
from jetbot import Camera, bgr8_to_jpeg  # JetBot用に用意したカメラと画像変換ライブラリを利用します。

########################################
# カメラを有効化します。
# 画像はwidthとheightで指定したピクセルサイズにリサイズされます。
########################################
camera = Camera.instance(width=224, height=224)

########################################
# 画像表示用のウィジェットを用意します。
# widthとheightは表示するウィジェットの幅と高さです。
# カメラ画像サイズと一致する必要はありません。
########################################
image = widgets.Image(format='jpeg', width=224, height=224)

########################################
# 「blocked」の確率を表示するためのスライダーを用意します。
########################################
blocked_slider = widgets.FloatSlider(description='blocked', min=0.0, max=1.0, orientation='vertical')

########################################
# traitletsライブラリを利用してカメラ画像データが更新されたときに、
# bgr8フォーマットをjpegフォーマットに変換してから
# 画像表示ウィジェットに反映するように設定します。
########################################
camera_link = traitlets.dlink((camera, 'value'), (image, 'value'), transform=bgr8_to_jpeg)

########################################
# 画像表示ウィジェットとスライダーをブラウザに表示します。
########################################
display(widgets.HBox([image, blocked_slider]))

モーターを制御するためにrobotインスタンスを生成します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
from jetbot import Robot  # JetBotを制御するためのライブラリを利用します。

########################################
# JetBotの制御用クラスをインスタンス化します。
########################################
robot = Robot()

次は、カメラの画像が更新されるたびに呼び出される関数を生成します。この関数は、下記ステップを実行します。

1. カメラ画像をPre-processingにかけてモデル入力データに変換する
2. モデル推論の実行
3. 推論結果が50%以上の確率で「blocked」の場合は、左に曲がります。それ以外の場合は前進します。

In [ ]:
########################################
# 利用するライブラリを読み込みます。
########################################
import torch.nn.functional as F
import time

########################################
# カメラ画像が更新されたときに実行する処理を定義します。
########################################
def update(change):
    # update()関数はメインスレッドで動作します。このglobal宣言された変数は同じメインスレッドで定義されているため、省略可能です。
    # update()関数をサブスレッドから呼び出す場合はglobal宣言が必要になります。
    global blocked_slider, robot
    # カメラ画像を変数xにコピーします。
    x = change['new'] 
    # カメラ画像をモデルの入力データに変換します。
    x = preprocess(x)
    # 推論を実行します。
    y = model(x)
    
    # softmax()関数を適用して出力ベクトルの合計が1になるように正規化します（これにより確率分布になります）
    y = F.softmax(y, dim=1)
    
    # 入力データは多次元のバッチ配列になっています。出力もそれに対応しているためyは多次元配列になっています。
    # y.flatten()を呼び出すことで可能な限り不要な次元を除去します。([[blocked_rate, free_rate]]を[blocked_rate, free_rate]に変換)
    # そのうえで、「blocked」の確率となるy.flatten()[0]の値を取得します。「free」の確率を取得する場合はy.flatten()[1]になります。
    prob_blocked = float(y.flatten()[0])
    
    # 「blocked」の確率をスライダーに反映します。
    blocked_slider.value = prob_blocked
    
    # 「blocked」の確率が50%未満なら直進します。それ以外は左に旋回します。
    if prob_blocked < 0.5:
        robot.forward(0.4)  # JetBotのモーター出力を40%にして前進します。
    else:
        robot.left(0.4)  # JetBotのモーター出力を40%にして左に旋回します。
    
    time.sleep(0.001)  # 値がモーター制御基板のICチップに反映されるまで少し待ちます。
        
update({'new': camera.value})  # update()関数を1回呼び出して初期化します。

モデル推論からJetBotの動作までを実行する関数を作成しました。  
今度はそれをカメラ画像の更新に連動して動作させる必要があります。

JetBotでは、traitlets.HasTraitsを継承したCameraクラスを実装しているので、observe()を呼び出すだけで実現できます。

> （注意！）このコードを実行するとJetBotが動作しはじめます。JetBotが机から落ちないように十分注意してください。衝突回避は機能するはずですが、結果は学習データの良し悪しに影響されます。

In [ ]:
camera.observe(update, names='value')  # Cameraクラスのtraitlets.Any()型のvalue変数をupdate()関数に接続します。

これでカメラ画像が更新されるごとにJetBotで推論が実行され、動作命令が生成されているはずです。JetBotを障害物回避のコースに置いて、その動作を確認することができます。

# JetBotの停止
JetBotを停止したい場合は、以下のコードを実行して、カメラ画像の更新と推論実行の接続を解除できます。

In [ ]:
import time  # すでに読込み済みのライブラリなので、ここでは省略できます。

# カメラ画像が更新されたときにupdate()関数を呼び出していた接続を解除します。
camera.unobserve(update, names='value')

# フレームの処理の完了を待つために、小さなスリープを追加します。
time.sleep(0.1)

# JetBotのモーターを停止します。
robot.stop()

画像表示ウィジェットでのカメラ映像の更新を停止したい場合は、以下のようにカメラとウィジェットを連動させているリンクを解除できます。

In [ ]:
camera_link.unlink()  # 画像表示ウィジェットでのカメラ映像の更新が止まりますが、JetBot本体でのカメラは動作し続けます。

再度画像表示ウィジェットでのカメラ映像を更新したい場合は、以下を実行してください。

In [ ]:
camera_link.link()  # 画像表示ウィジェットでのカメラ映像の更新が再開されます。カメラ自体の動作は変更しないため、もしカメラが停止している場合は映像は更新されません。

繰り返しになりますが、他のノートブックでカメラを使用できるように、ここで利用したカメラのプロセスを適切に終了してください。

In [ ]:
camera.stop()

### 結論
このライブデモは以上です。うまくいけばあなたのJetBotは賢く衝突を避けていることでしょう。

collision avoidanceが上手く行かない場合、正しく走行できるように失敗しやすい場所でさらにデータを追加してください。  
このようにうまくいかない場所を中心にデータを収集すれば、JetBotはさらによく動作するはずです。